# spherical_demo.ipynb
# WESmith 06/29/22

In [1]:
import operator_utils as ws
import sympy as sy

# 3D spherical coordinates

In [2]:
aa = ws.Coords()
dd = ws.Operators(aa.spherical)

In [3]:
for k in dd.transform: ws.pp(k)

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [4]:
dd.Z

Matrix([
[\hat{\mathbf{e_x}}],
[\hat{\mathbf{e_y}}],
[\hat{\mathbf{e_z}}]])

In [5]:
dd.Zp

Matrix([
[     \hat{\mathbf{e_r}}],
[\hat{\mathbf{e_\theta}}],
[  \hat{\mathbf{e_\phi}}]])

In [6]:
dd.Jacobian

Matrix([
[sin(theta)*cos(phi), r*cos(phi)*cos(theta), -r*sin(phi)*sin(theta)],
[sin(phi)*sin(theta), r*sin(phi)*cos(theta),  r*sin(theta)*cos(phi)],
[         cos(theta),         -r*sin(theta),                      0]])

In [7]:
ws.pp(dd.Jacobian_inv)

<IPython.core.display.Math object>

In [8]:
ws.pp(dd.primed_basis)

<IPython.core.display.Math object>

In [9]:
dd.metric

Matrix([
[1,    0,                  0],
[0, r**2,                  0],
[0,    0, r**2*sin(theta)**2]])

In [10]:
dd.metric_sqrt

Matrix([
[1, 0,            0],
[0, r,            0],
[0, 0, r*sin(theta)]])

In [11]:
dd.metric_inv_sqrt

Matrix([
[1,   0,                0],
[0, 1/r,                0],
[0,   0, 1/(r*sin(theta))]])

In [12]:
dd.det_sqrt

r**2*sin(theta)

In [13]:
dd.gradient()

Matrix([
[                 Derivative(f(r, theta, phi), r)],
[           Derivative(f(r, theta, phi), theta)/r],
[Derivative(f(r, theta, phi), phi)/(r*sin(theta))]])

In [14]:
dd.gradient(vector=False)

Matrix([[\hat{\mathbf{e_\phi}}*Derivative(f(r, theta, phi), phi)/(r*sin(theta)) + \hat{\mathbf{e_\theta}}*Derivative(f(r, theta, phi), theta)/r + \hat{\mathbf{e_r}}*Derivative(f(r, theta, phi), r)]])

In [15]:
dd.divergence()

Derivative(A_r(r, theta, phi), r) + A_\theta(r, theta, phi)*cos(theta)/(r*sin(theta)) + 2*A_r(r, theta, phi)/r + Derivative(A_\theta(r, theta, phi), theta)/r + Derivative(A_\phi(r, theta, phi), phi)/(r*sin(theta))

In [16]:
dd.curl()

Matrix([
[A_\phi(r, theta, phi)*cos(theta)/(r*sin(theta)) + Derivative(A_\phi(r, theta, phi), theta)/r - Derivative(A_\theta(r, theta, phi), phi)/(r*sin(theta))],
[                                  -Derivative(A_\phi(r, theta, phi), r) - A_\phi(r, theta, phi)/r + Derivative(A_r(r, theta, phi), phi)/(r*sin(theta))],
[                                          Derivative(A_\theta(r, theta, phi), r) + A_\theta(r, theta, phi)/r - Derivative(A_r(r, theta, phi), theta)/r]])

In [17]:
dd.curl(vector=False)

Matrix([[\hat{\mathbf{e_\phi}}*(Derivative(A_\theta(r, theta, phi), r) + A_\theta(r, theta, phi)/r - Derivative(A_r(r, theta, phi), theta)/r) + \hat{\mathbf{e_\theta}}*(-Derivative(A_\phi(r, theta, phi), r) - A_\phi(r, theta, phi)/r + Derivative(A_r(r, theta, phi), phi)/(r*sin(theta))) + \hat{\mathbf{e_r}}*(A_\phi(r, theta, phi)*cos(theta)/(r*sin(theta)) + Derivative(A_\phi(r, theta, phi), theta)/r - Derivative(A_\theta(r, theta, phi), phi)/(r*sin(theta)))]])

In [18]:
dd.Laplacian_scalar()  # of a general function

Derivative(f(r, theta, phi), (r, 2)) + 2*Derivative(f(r, theta, phi), r)/r + Derivative(f(r, theta, phi), (theta, 2))/r**2 + Derivative(f(r, theta, phi), theta)/(r**2*tan(theta)) + Derivative(f(r, theta, phi), (phi, 2))/(r**2*sin(theta)**2)

In [19]:
r, theta, phi = dd.coords_p

In [20]:
# separable function
f, g, h, F = sy.symbols('f g h F', cls=sy.Function)
f = f(r)
g = g(theta)
h = h(phi)
F = f * g * h

In [21]:
dd.gradient(F)

Matrix([
[                 g(theta)*h(phi)*Derivative(f(r), r)],
[           f(r)*h(phi)*Derivative(g(theta), theta)/r],
[f(r)*g(theta)*Derivative(h(phi), phi)/(r*sin(theta))]])

In [22]:
dd.Laplacian_scalar(F)

g(theta)*h(phi)*Derivative(f(r), (r, 2)) + 2*g(theta)*h(phi)*Derivative(f(r), r)/r + f(r)*g(theta)*Derivative(h(phi), (phi, 2))/(r**2*sin(theta)**2) + f(r)*h(phi)*Derivative(g(theta), (theta, 2))/r**2 + f(r)*h(phi)*Derivative(g(theta), theta)/(r**2*tan(theta))

In [23]:
# Laplacian of f(r) only
dd.Laplacian_scalar(f)

Derivative(f(r), (r, 2)) + 2*Derivative(f(r), r)/r

In [24]:
dd.Laplacian_scalar(1/f)  # this works

-Derivative(f(r), (r, 2))/f(r)**2 + 2*Derivative(f(r), r)**2/f(r)**3 - 2*Derivative(f(r), r)/(r*f(r)**2)

In [25]:
dd.divergence(dd.gradient(g*h)) # also the Laplacian: note that products of functions work

g(theta)*Derivative(h(phi), (phi, 2))/(r**2*sin(theta)**2) + h(phi)*Derivative(g(theta), (theta, 2))/r**2 + h(phi)*cos(theta)*Derivative(g(theta), theta)/(r**2*sin(theta))

In [26]:
dd.Laplacian_vector_full(dd.gradient(h))

Matrix([
[           -2*Derivative(h(phi), (phi, 2))/(r**3*sin(theta)**2)],
[-2*cos(theta)*Derivative(h(phi), (phi, 2))/(r**3*sin(theta)**3)],
[              Derivative(h(phi), (phi, 3))/(r**3*sin(theta)**3)]])

In [27]:
vv = dd.Laplacian_vector_full()

In [28]:
vv[0]

Derivative(A_r(r, theta, phi), (r, 2)) + 2*Derivative(A_r(r, theta, phi), r)/r - 2*A_\theta(r, theta, phi)/(r**2*tan(theta)) - 2*A_r(r, theta, phi)/r**2 - 2*Derivative(A_\theta(r, theta, phi), theta)/r**2 + Derivative(A_r(r, theta, phi), (theta, 2))/r**2 + Derivative(A_r(r, theta, phi), theta)/(r**2*tan(theta)) - 2*Derivative(A_\phi(r, theta, phi), phi)/(r**2*sin(theta)) + Derivative(A_r(r, theta, phi), (phi, 2))/(r**2*sin(theta)**2)

In [29]:
# successfully validated against Arfken p.82
vr = dd.Laplacian_vector_reduced()

In [30]:
vr[0]

\nabla^{2}(A_r(r, theta, phi)) - 2*A_\theta(r, theta, phi)/(r**2*tan(theta)) - 2*A_r(r, theta, phi)/r**2 - 2*Derivative(A_\theta(r, theta, phi), theta)/r**2 - 2*Derivative(A_\phi(r, theta, phi), phi)/(r**2*sin(theta))

In [31]:
vr[1]

\nabla^{2}(A_\theta(r, theta, phi)) - A_\theta(r, theta, phi)/(r**2*sin(theta)**2) + 2*Derivative(A_r(r, theta, phi), theta)/r**2 - 2*cos(theta)*Derivative(A_\phi(r, theta, phi), phi)/(r**2*sin(theta)**2)

In [32]:
vr[2]

\nabla^{2}(A_\phi(r, theta, phi)) - A_\phi(r, theta, phi)/(r**2*sin(theta)**2) + 2*Derivative(A_r(r, theta, phi), phi)/(r**2*sin(theta)) + 2*cos(theta)*Derivative(A_\theta(r, theta, phi), phi)/(r**2*sin(theta)**2)

In [33]:
dd.material_derivative(vector=False)

Matrix([[\hat{\mathbf{e_\phi}}*(A_r(r, theta, phi)*Derivative(B_\phi(r, theta, phi), r) + A_\phi(r, theta, phi)*B_\theta(r, theta, phi)*cos(theta)/(r*sin(theta)) + A_\phi(r, theta, phi)*B_r(r, theta, phi)/r + A_\phi(r, theta, phi)*Derivative(B_\phi(r, theta, phi), phi)/(r*sin(theta)) + A_\theta(r, theta, phi)*Derivative(B_\phi(r, theta, phi), theta)/r) + \hat{\mathbf{e_\theta}}*(A_r(r, theta, phi)*Derivative(B_\theta(r, theta, phi), r) - A_\phi(r, theta, phi)*B_\phi(r, theta, phi)*cos(theta)/(r*sin(theta)) + A_\phi(r, theta, phi)*Derivative(B_\theta(r, theta, phi), phi)/(r*sin(theta)) + A_\theta(r, theta, phi)*B_r(r, theta, phi)/r + A_\theta(r, theta, phi)*Derivative(B_\theta(r, theta, phi), theta)/r) + \hat{\mathbf{e_r}}*(A_r(r, theta, phi)*Derivative(B_r(r, theta, phi), r) - A_\phi(r, theta, phi)*B_\phi(r, theta, phi)/r + A_\phi(r, theta, phi)*Derivative(B_r(r, theta, phi), phi)/(r*sin(theta)) - A_\theta(r, theta, phi)*B_\theta(r, theta, phi)/r + A_\theta(r, theta, phi)*Derivative(B_

In [34]:
dd.curl(dd.curl())

Matrix([
[Derivative(A_\theta(r, theta, phi), r, theta)/r + cos(theta)*Derivative(A_\theta(r, theta, phi), r)/(r*sin(theta)) + Derivative(A_\phi(r, theta, phi), phi, r)/(r*sin(theta)) + A_\theta(r, theta, phi)*cos(theta)/(r**2*sin(theta)) + Derivative(A_\theta(r, theta, phi), theta)/r**2 - Derivative(A_r(r, theta, phi), (theta, 2))/r**2 - cos(theta)*Derivative(A_r(r, theta, phi), theta)/(r**2*sin(theta)) + Derivative(A_\phi(r, theta, phi), phi)/(r**2*sin(theta)) - Derivative(A_r(r, theta, phi), (phi, 2))/(r**2*sin(theta)**2)],
[                                                                                                                                                                                  -Derivative(A_\theta(r, theta, phi), (r, 2)) - 2*Derivative(A_\theta(r, theta, phi), r)/r + Derivative(A_r(r, theta, phi), r, theta)/r + Derivative(A_\phi(r, theta, phi), phi, theta)/(r**2*sin(theta)) + cos(theta)*Derivative(A_\phi(r, theta, phi), phi)/(r**2*sin(theta)**2) - Derivative(A

In [35]:
dd.gradient(dd.divergence())

Matrix([
[   Derivative(A_r(r, theta, phi), (r, 2)) + 2*Derivative(A_r(r, theta, phi), r)/r + Derivative(A_\theta(r, theta, phi), r, theta)/r + cos(theta)*Derivative(A_\theta(r, theta, phi), r)/(r*sin(theta)) + Derivative(A_\phi(r, theta, phi), phi, r)/(r*sin(theta)) - A_\theta(r, theta, phi)*cos(theta)/(r**2*sin(theta)) - 2*A_r(r, theta, phi)/r**2 - Derivative(A_\theta(r, theta, phi), theta)/r**2 - Derivative(A_\phi(r, theta, phi), phi)/(r**2*sin(theta))],
[Derivative(A_r(r, theta, phi), r, theta)/r - A_\theta(r, theta, phi)/r**2 - A_\theta(r, theta, phi)*cos(theta)**2/(r**2*sin(theta)**2) + Derivative(A_\theta(r, theta, phi), (theta, 2))/r**2 + 2*Derivative(A_r(r, theta, phi), theta)/r**2 + cos(theta)*Derivative(A_\theta(r, theta, phi), theta)/(r**2*sin(theta)) + Derivative(A_\phi(r, theta, phi), phi, theta)/(r**2*sin(theta)) - cos(theta)*Derivative(A_\phi(r, theta, phi), phi)/(r**2*sin(theta)**2)],
[                                                                                    

In [36]:
def Arfken_example_2_4_2(ops):
    # ops is an Operators object defined with standard spherical coords r, theta, phi
    # this was successfully validated against Arfken result p. 83 6/18/22
    r, theta, phi = ops.coords_p
    Aphi = sy.symbols('A_phi', cls=sy.Function, real=True)
    Aphi = Aphi(r, theta)
    A    = sy.Matrix([0, 0, Aphi])
    v1   = ops.curl(ops.curl(A))
    tmp  = []
    for j, k in zip(v1, A):
        tmp.append(j + ops.Laplacian_scalar(k) - ops.Laplacian_symbolic(k))
    reduced = sy.trigsimp(ops.Zp.T * sy.Matrix(tmp))
    full    = sy.trigsimp(ops.Zp.T * v1)
    return full, reduced

In [37]:
full, reduced = Arfken_example_2_4_2(dd)
full

Matrix([[\hat{\mathbf{e_\phi}}*(-Derivative(A_phi(r, theta), (r, 2)) - 2*Derivative(A_phi(r, theta), r)/r + A_phi(r, theta)/(r**2*sin(theta)**2) - Derivative(A_phi(r, theta), (theta, 2))/r**2 - Derivative(A_phi(r, theta), theta)/(r**2*tan(theta)))]])

In [38]:
reduced

Matrix([[\hat{\mathbf{e_\phi}}*(-\nabla^{2}(A_phi(r, theta)) + A_phi(r, theta)/(r**2*sin(theta)**2))]])

# Basic checks

In [39]:
dd.Laplacian_scalar(1/r)  # should be 0

0

In [40]:
dd.gradient(1/r, vector=False)  # should be -1/r**2 in the r_hat direction

Matrix([[-\hat{\mathbf{e_r}}/r**2]])

In [63]:
xx = dd.curl(dd.gradient())  # should be 0 vector

In [64]:
xx

Matrix([
[0],
[0],
[0]])

In [65]:
#(xx.T * xx).args[2][0]
xx._eval_is_zero_matrix()

True

In [66]:
dd.divergence(dd.curl())  # should be 0

0